## Created By: Prateek Kakkar and Ramgopal Reddy Putta
### Contact Information: pkakkar@seattleu.edu and rputta@seattleu.edu

### Install Libraries

In [1]:
!pip install -Uqq pypdfium2
!pip install -Uqq tabula-py
!pip install -Uqq pdfplumber
!pip install -Uqq python-dateutil
!pip install -Uqq xlrd

!pip install -Uqq regex
!pip install -Uqq camelot-py[cv]

### Load Libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pypdfium2 as pdfium
import pandas as pd
import numpy as np
import os
import os.path
import tabula
from tabula.io import read_pdf
import pdfplumber
import re

from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

import uuid
import camelot as cam
from numpy.core.fromnumeric import transpose
from pathlib import Path
import tkinter

### Define Functions

#### Tera Nova Extract Text Fields

In [3]:
#### Tera Nova Extract Text Fields# This function takes pdf as an input
# converts into text
# returns the required fields
    
def TeraNovaText(folder,file,prjct_id,scrpr_id):
    pdf_fium = pdfium.PdfDocument(folder+'/'+file) # Read the PDF as a text File
    pdf_plumb = pdfplumber.open(folder+'/'+file) # Read the PDF as a text File
    
    no_of_pages = len(pdf_fium)
    
    out= []  # to store the output of this function
    
    for i in range(no_of_pages):
        temp_out = []
        
        page_fium = pdf_fium[i] # pdfium
        page_plumb = pdf_plumb.pages[i] # PDF Plumber
        
        # Load a text page helper
        textpage_fium = page_fium.get_textpage() # pdfium
    
        # Extract text from the whole page
        text_fium = textpage_fium.get_text_range() # pdfium
        text_plumb = page_plumb.extract_text() # PDF Plumber
    
        # Split the list with the delimiters
        list_fium = text_fium.split('\r\n') # pdfium
        list_plumb = text_plumb.split('\n')  # PDF Plumber
    
    
        # Store the requird fields
    
        # Data Ingestion_datetime
        Ingestion_datetime = pd.to_datetime('today').strftime("%m/%d/%Y %I:%M:%S %p")
    
        # City
        #City = list_plumb[3].split(',')[0].split(' ')[-1]
    
        # DATE MOLDED
        date_molded_src = "DATE MOLDED"
        date_molded_src_get_string = [x for x in list_fium if date_molded_src in x]
        date_molded_src_lst_str = ''.join(date_molded_src_get_string)
        Date_Molded = date_molded_src_lst_str.split(' ')[2]
    
        # DATE ISSUED
        date_issue_src = "DATE ISSUED"
        date_issue_src_get_string = [x for x in list_fium if date_issue_src in x]
        date_issue_src_lst_str = ''.join(date_issue_src_get_string)
        Date_Issued = date_issue_src_lst_str.split(':')[1].strip()
    
        # LAB NUMBER
        Lab_Number = date_molded_src_lst_str.split(' ')[5]
    
        # LOCATION OF PLACEMENT
        colon = ":"
        lop_searc = "LOCATION OF PLACEMENT"
        lop_searc_get_string = [x for x in list_fium if lop_searc in x]
        lop_searc_lst_str = ''.join(lop_searc_get_string)

        loc = lop_searc_lst_str

        if any(c in colon for c in loc):
            Location_of_Placement = loc.split(':')[1].strip()
        else:
            loc = loc.split(' ')
            loc.insert(3, ':')
            loc = " ".join(loc) 
            Location_of_Placement = loc.split(':')[1].strip()
        
        # CONCRETE SUPPLIER
        supplier_src = "CONCRETE SUPPLIER"
        supplier_src_get_string = [x for x in list_plumb if supplier_src in x]
        supplier_src_lst_str = ''.join(supplier_src_get_string)
        Concrete_Supplier = supplier_src_lst_str.split('WATER')[0].split(':')[1].strip()
        
        # MIX ID NO
        mixid_src = "MIX ID NO"
        mixid_src_get_string = [x for x in list_plumb if mixid_src in x]
        mixid_src_lst_str = ''.join(mixid_src_get_string)
        Mix_ID_No = mixid_src_lst_str.split('UNIT')[0].split(':')[1].strip()
    
        # WEATHER
        weather_src = "WEATHER"
        weather_src_get_string = [x for x in list_fium if weather_src in x]
        if weather_src_get_string:
            weather_src_lst_str = ''.join(weather_src_get_string)
            Weather = weather_src_lst_str.split(':')[1].strip()
            
        else:
            weather_src = "W EATHER"
            weather_src_get_string = [x for x in list_fium if weather_src in x]
            weather_src_lst_str = ''.join(weather_src_get_string)
            Weather = weather_src_lst_str.split(':')[1].strip()
    
        # TIME MOLDED
        time_src = "TIME MOLDED"
        time_src_get_string = [x for x in list_plumb if time_src in x]
        time_src_lst_str = ''.join(time_src_get_string)
        Time_Molded = time_src_lst_str.split(' ')[2] + ' ' +time_src_lst_str.split(' ')[3]
        if "ASTM" in Time_Molded:
            Time_Molded = ""
        else:
            Time_Molded = Time_Molded[0:8]
    
        # AIR CONTENT(%)
        aircontent_src = "AIR CONTENT"
        aircontent_src_get_string = [x for x in list_fium if aircontent_src in x]
        aircontent_src_lst_str = ''.join(aircontent_src_get_string)
        Air_Content = aircontent_src_lst_str.split(':')[1].strip()
        Air_Content = re.sub('[^0-9,.]', '', Air_Content)
    
        # SLUMP(IN)
        slump_src = "SLUMP(IN)"
        slump_src_get_string = [x for x in list_fium if slump_src in x]
        slump_src_lst_str = ''.join(slump_src_get_string)
        Slump_space_chck = slump_src_lst_str.split(':')
        if len(Slump_space_chck[1])>5:
            Slump = Slump_space_chck[1].split(' ')[1].strip()
        else:
            Slump = Slump_space_chck[1].strip()
        
        Slump = re.sub('[^0-9,.]', '', Slump)
        
    
        # SIZE & REQUIRED PSI 
        size_psi_search = "SIZE:"
        size_psi_get_string = [x for x in list_plumb if size_psi_search in x]
        if size_psi_get_string:
            size_psi_lst_str = ''.join(size_psi_get_string)
            Size = size_psi_lst_str.split('DIAMETER')[0].split(':')[-1].strip()
            Required_PSI = size_psi_lst_str.split(' ')[-1].strip()
        else:
            size_psi_search = "SIZE"
            size_psi_get_string = [x for x in list_plumb if size_psi_search in x]
            size_psi_lst_str = ''.join(size_psi_get_string)
            Size = size_psi_lst_str.split('DIAMETER')[0].split('SIZE')[-1].strip()
            Required_PSI = size_psi_lst_str.split(' ')[-1].strip()
        
        # WATER ADDED(GALS)    
        water_searc = "GALS"
        water_searc_get_string = [x for x in list_fium if water_searc in x]
        water_searc_lst_str = ''.join(water_searc_get_string)
        Water_Added_len_check = water_searc_lst_str.split(':')
        
        if len(Water_Added_len_check[1])>5:
            Water_Added = Water_Added_len_check[1].split(' ')[1].strip()
        else:
            Water_Added = Water_Added_len_check[1].strip()
        
        Water_Added = re.sub('[^0-9,.]', '', Water_Added)
    
    
        # UNIT WEIGHT(PCF):
        unit_searc = "PCF"
        unit_searc_get_string = [x for x in list_fium if unit_searc in x]
        unit_searc_lst_str = ''.join(unit_searc_get_string)
        Unit_Weight = unit_searc_lst_str.split(':')[1].strip()
        Unit_Weight = re.sub('[^0-9,.]', '', Unit_Weight)
    
        # AMBIENT TEMP(F):
        ambi_temp_searc = "AMBIENT TEMP"
        ambi_temp_searc_get_string = [x for x in list_fium if ambi_temp_searc in x]
        ambi_temp_searc_lst_str = ''.join(ambi_temp_searc_get_string)
        Ambient_Temp = ambi_temp_searc_lst_str.split(':')[1].strip()
        Ambient_Temp = re.sub('[^0-9]', '', Ambient_Temp)
    
        # CONCRETE TEMP(F):
        concrete_temp_searc = "CONCRETE TEMP"
        concrete_temp_searc_get_string = [x for x in list_fium if concrete_temp_searc in x]
        concrete_temp_searc_lst_str = ''.join(concrete_temp_searc_get_string)
        Concrete_Temp = concrete_temp_searc_lst_str.split(':')[1].strip()
        Concrete_Temp = re.sub('[^0-9]', '', Concrete_Temp)
        
        # PROJECT ID
        ProjectID = prjct_id
        
        # SCRAPER ID
        ScraperID = scrpr_id
        
        temp_out = [ProjectID,ScraperID,file,Ingestion_datetime,Date_Molded,Date_Issued,Lab_Number,Location_of_Placement
       ,Concrete_Supplier,Mix_ID_No,Weather,Time_Molded,Air_Content,Slump,Size
       ,Required_PSI,Water_Added,Unit_Weight,Ambient_Temp,Concrete_Temp]
        
        out.append((temp_out))
    
    return out


#### Tera Nova Extract Test Report Table

In [4]:
# This function takes pdf as an input
# converts into a dataframe
# returns the test results as a DataFrame

def TeraNovaPdf2Table(folder,file,result):
    readpdf2df = tabula.io.read_pdf(folder+'/'+file 
                             , pages='all'
                             #,output_format="dataframe"
                                   , multiple_tables=True)
    
    df = pd.concat(readpdf2df)
    age_load_read = df.iloc[1:,3].dropna().reset_index().drop(columns=['index']).iloc[:-1,]
    
    drp_extra = age_load_read[age_load_read.iloc[:,0].str.contains('diameter', case=False, na=False)].index
    age_load_read = age_load_read.drop(drp_extra)
    drp_extra = age_load_read[age_load_read.iloc[:,0].str.contains('compression', case=False, na=False)].index
    age_load_read = age_load_read.drop(drp_extra)
    drp_extra = age_load_read[age_load_read.iloc[:,0].str.contains('age', case=False, na=False)].index
    age_load_read = age_load_read.drop(drp_extra)
    
    # Extra Space in the Data Check
    space_check = age_load_read.iloc[:,0].str.split(' ').tolist()
    
    space_check_ref = []
    for i in space_check:
        for j in i:
            if len(i)>4:
                i[1]=i[1]+i[2]
                i.pop(2)
        space_check_ref.append(i)
    space_check = space_check_ref
    age_load_read = pd.DataFrame(space_check, columns =['Age_Days','Date_Tested','Total_load', 'Unit_load'])
    final_df = pd.DataFrame()
    for i in range(len(result)):
        df_split = np.array_split(age_load_read, len(result))
        
        result_lab = result[i][6]
        result_date_isssue = result[i][5]
        
        df_sel = df_split[i]
        
        iterat = df_sel.shape[0]
        
        lab_nbr = pd.DataFrame({'Lab_Number': result_lab}, index=[0])
        date_isu = pd.DataFrame({'Date_Issued': result_date_isssue}, index=[0])
        
        lab_nbr_itr = pd.DataFrame(np.repeat(lab_nbr.values, iterat, axis=0))
        date_isu_itr = pd.DataFrame(np.repeat(date_isu.values, iterat, axis=0))
        
        lab_nbr_itr.columns = lab_nbr.columns
        date_isu_itr.columns = date_isu.columns
        
        temp_df = pd.concat([lab_nbr_itr,date_isu_itr,age_load_read.iloc[:, age_load_read.columns != 'Total_load']], axis=1, join='inner')
        final_df = final_df.append(temp_df).reset_index(drop=True)
        
    
    return final_df
    
    
    

#### TeraNova Main Function

In [5]:
def TeraNova(folder_dir,prjct_id,scrpr_id):
    log = []
    df_text = []
    pdf_df = pd.DataFrame()

    #path = '/Users/prateek/Documents/Captsone Project/Jupiter Files/TestFolder'
    
    #os.chdir('C:/Users/pkakkar/Documents/2023-02-17-DataSet-006-008')
    #path = 'C:/Users/pkakkar/Documents/2023-02-17-DataSet-006-008'
    
    os.chdir(folder_dir)
    path = folder_dir


    for root, dirs, files in os.walk(path,topdown=True):
        if not root.endswith('.DS_Store'):
            for file in files:
                if file.endswith('.pdf'):
                    read_datetime = pd.to_datetime('today').strftime("%m/%d/%Y %I:%M:%S %p")
                    folder = os.path.basename(root)
                    dir_path = os.path.dirname(root)
                
                    log.append((prjct_id,scrpr_id,dir_path,folder,file,read_datetime))  # Appends results for log DF
                
                    result = TeraNovaText(root,file,prjct_id,scrpr_id) # Calls text function
                
                    for j in range(len(result)):
                        df_text.append((result[j]))    # Appends results from text function
                     
                    df_out = TeraNovaPdf2Table(root,file,result) # Calls Dataframe function
                    pdf_df = pdf_df.append(df_out).reset_index(drop=True) # Appends test results DF
                

    # Append Log DataFrame        
    df_log = pd.DataFrame(log, columns=['ProjectID', 'ScraperID','Directory','Folder','File', 'Read_DateTime'])

    # Append DataFrame with Text fields
    df_page = pd.DataFrame(df_text, columns=['ProjectID',
                     'ReportScraperID',
                     'File_Name',
                     'Ingestion_datetime',
                     'Date_Molded',
                     'Date_Issued',
                     'Lab_Number',
                     'Location_of_Placement',
                     'Concrete_Supplier',
                     'Mix_ID_No',
                     'Weather',
                     'Time_Molded',
                     'Air_Content(%)',
                     'Slump(in)',
                     'Size',
                     'Required_PSI',
                     'Water_Added(GALS)',
                     'Unit_Weight(PCF)',
                     'Ambient_Temp(F)',
                     'Concrete_Temp(F)'])
    
    return df_page,pdf_df,df_log

#### Tera Nova Data Transformation

In [6]:
def TeraNovaDataTransform(TeraNovaFieldReport, TeraNovaTestResult):
    ############## Create a copy of the Dataframes  #############

    TeraNovaFieldReport_stg = TeraNovaFieldReport.copy()
    TeraNovaTestResult_stg = TeraNovaTestResult.copy()
    
    # generate the uuid for BatchLabID 
    batchlabID = {Lab_Number: str(uuid.uuid4()) for Lab_Number in TeraNovaFieldReport_stg['Lab_Number'].unique()}
    
    # map uuid
    TeraNovaFieldReport_stg['BatchLabID'] = TeraNovaFieldReport_stg['Lab_Number'].map(batchlabID)
    
    # Add BatchLabID to Results Table
    TeraNovaTestResult_stg['BatchLabID'] = TeraNovaTestResult_stg['Lab_Number'].map(batchlabID)
    
    
    ###################### Rename the Columns ########################

    TeraNovaFieldReport_stg = TeraNovaFieldReport_stg.rename(columns={'File_Name': 'ReportFileName'
                                              , 'Date_Issued': 'ReportDateIssued'
                                              , 'Ingestion_datetime': 'ReportTimeIngested'
                                              , 'Ambient_Temp(F)': 'SiteTemperature'
                                              , 'Weather': 'SiteWeather'
                                              , 'Concrete_Supplier': 'ConcreteSupplier'
                                              , 'Location_of_Placement': 'ConcretePlacementLocation'
                                              , 'Lab_Number': 'BatchLabNumber'
                                              , 'Mix_ID_No': 'BatchMixID'
                                              , 'Size': 'BatchSpecimenSize'
                                              , 'Concrete_Temp(F)': 'BatchTemperature'
                                              , 'Unit_Weight(PCF)': 'BatchUnitWeight'
                                              , 'Required_PSI': 'BatchRequiredStrength'
                                              , 'Air_Content(%)': 'BatchAirContent'
                                              , 'Slump(in)': 'BatchtSlump'
                                              , 'Water_Added(GALS)': 'BatchWaterAdded'})



    TeraNovaTestResult_stg = TeraNovaTestResult_stg.rename(columns={'Lab_Number': 'BatchLabNumber'
                                              , 'Date_Issued': 'ReportDateIssued'
                                              , 'Date_Tested': 'SpecimenDateTested'
                                              , 'Unit_load': 'SpecimenMeasuredStrength'
                                              , 'Age_Days': 'SpecimenAgeTested'})



    ########### Changing Data Types of Columns for SpecimenFieldReport #########

    TeraNovaFieldReport_stg['ReportTimeIngested'] = TeraNovaFieldReport_stg['ReportTimeIngested'].astype('datetime64[ns]')
    TeraNovaFieldReport_stg['ReportDateIssued'] = pd.to_datetime(TeraNovaFieldReport_stg["ReportDateIssued"], format="%d-%b-%y")


    TeraNovaFieldReport_stg['SiteTemperature'] = TeraNovaFieldReport_stg['SiteTemperature'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchTemperature'] = TeraNovaFieldReport_stg['BatchTemperature'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchUnitWeight'] = TeraNovaFieldReport_stg['BatchUnitWeight'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchRequiredStrength'] = TeraNovaFieldReport_stg['BatchRequiredStrength'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchAirContent'] = TeraNovaFieldReport_stg['BatchAirContent'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchtSlump'] = TeraNovaFieldReport_stg['BatchtSlump'].replace(r'^\s*$', np.nan, regex=True)
    TeraNovaFieldReport_stg['BatchWaterAdded'] = TeraNovaFieldReport_stg['BatchWaterAdded'].replace(r'^\s*$', np.nan, regex=True)

    #df_page_stg['BatchLabNumber'] = df_page_stg['BatchLabNumber'].astype('int')
    TeraNovaFieldReport_stg['SiteTemperature'] = TeraNovaFieldReport_stg['SiteTemperature'].astype('float')
    TeraNovaFieldReport_stg['BatchTemperature'] = TeraNovaFieldReport_stg['BatchTemperature'].astype('float')
    TeraNovaFieldReport_stg['BatchUnitWeight'] = TeraNovaFieldReport_stg['BatchUnitWeight'].astype('float')
    TeraNovaFieldReport_stg['BatchRequiredStrength'] = TeraNovaFieldReport_stg['BatchRequiredStrength'].astype('float')
    TeraNovaFieldReport_stg['BatchAirContent'] = TeraNovaFieldReport_stg['BatchAirContent'].astype('float')
    TeraNovaFieldReport_stg['BatchtSlump'] = TeraNovaFieldReport_stg['BatchtSlump'].astype('float')
    TeraNovaFieldReport_stg['BatchWaterAdded'] = TeraNovaFieldReport_stg['BatchWaterAdded'].astype('float')
    
    TeraNovaFieldReport_stg['BatchSpecimenSize'] = TeraNovaFieldReport_stg['BatchSpecimenSize'].str.replace(' ','')


    TeraNovaFieldReport_stg["Time_Molded"] = TeraNovaFieldReport_stg['Time_Molded'].str.lower()
    TeraNovaFieldReport_stg['Time_Molded'] = TeraNovaFieldReport_stg['Time_Molded'].replace({'sm':'am', 'om':'pm'}, regex = True)


    TeraNovaFieldReport_stg["BatchTimeMolded"] = TeraNovaFieldReport_stg["Date_Molded"] + ' ' +TeraNovaFieldReport_stg["Time_Molded"]
    TeraNovaFieldReport_stg['BatchTimeMolded'] = TeraNovaFieldReport_stg['BatchTimeMolded'].astype('datetime64[ns]')



    ########### Changing Data Types of Columns for SpecimenCompressionTestResult #########

    # Remove rows with not integer age days
    TeraNovaTestResult_stg = TeraNovaTestResult_stg[pd.to_numeric(TeraNovaTestResult_stg['SpecimenAgeTested'], errors='coerce').notna()]

    # Change ReportedDateIssues to datetime
    TeraNovaTestResult_stg['ReportDateIssued'] = pd.to_datetime(TeraNovaTestResult_stg["ReportDateIssued"], format="%d-%b-%y")

    # Create a new dataframe to handle junk datevalue
    junk_df = pd.DataFrame()

    # This code chnages SpecimenDateTested to datatime format
    for i in TeraNovaTestResult_stg['SpecimenDateTested']:
        if len(i)!= 9:
            junk_df = junk_df.append(TeraNovaTestResult_stg.loc[TeraNovaTestResult_stg['SpecimenDateTested'] == i]).reset_index(drop=True)
            TeraNovaTestResult_stg = TeraNovaTestResult_stg.drop(labels=[TeraNovaTestResult_stg.loc[TeraNovaTestResult_stg['SpecimenDateTested'] == i].index.values[0]]
                                         , axis=0).reset_index(drop=True)

    TeraNovaTestResult_stg['SpecimenDateTested'] = pd.to_datetime(TeraNovaTestResult_stg["SpecimenDateTested"], format="%d-%b-%y")

    if junk_df.empty:
        pass
    else:
        junk_df['SpecimenDateTested'] = pd.to_datetime(junk_df["SpecimenDateTested"], format="%d-%m-%y")
        TeraNovaTestResult_stg = pd.concat([TeraNovaTestResult_stg, junk_df], ignore_index=True, sort=False)

    del junk_df   # Delete Junk dataframe to release memory


    # Convert BatchLabNumber to int
    #TeraNovaTestResult_stg['BatchLabNumber'] = TeraNovaTestResult_stg['BatchLabNumber'].astype('int')

    # Convert SpecimentAgeTest to integer
    TeraNovaTestResult_stg['SpecimenAgeTested'] = TeraNovaTestResult_stg['SpecimenAgeTested'].astype('int')


    # Remove the outbound rows
    TeraNovaTestResult_stg = TeraNovaTestResult_stg[(TeraNovaTestResult_stg['SpecimenAgeTested'] >= 1) & (TeraNovaTestResult_stg['SpecimenAgeTested'] <= age_day_max)]



    #################################### Save only the Latest Data ############################################

    TeraNovaFieldReport_stg  = TeraNovaFieldReport_stg[TeraNovaFieldReport_stg.groupby('BatchLabNumber')['ReportDateIssued'].transform('max') == TeraNovaFieldReport_stg['ReportDateIssued']].reset_index(drop=True)

    TeraNovaTestResult_stg  = TeraNovaTestResult_stg[TeraNovaTestResult_stg.groupby('BatchLabNumber')['ReportDateIssued'].transform('max') == TeraNovaTestResult_stg['ReportDateIssued']].reset_index(drop=True)

    # Drop NA values from the compression test report
    TeraNovaTestResult_stg = TeraNovaTestResult_stg.dropna()

    # Convert SpecimenMeasuredStrength to integer
    TeraNovaTestResult_stg['SpecimenMeasuredStrength'] = TeraNovaTestResult_stg['SpecimenMeasuredStrength'].astype('int')

    # Add a new column called CylinderTestID
    TeraNovaTestResult_stg['CylinderTestID'] = TeraNovaTestResult_stg.sort_values(['ReportDateIssued','SpecimenAgeTested'], ascending=[True,True]) \
                 .groupby(['BatchLabNumber']) \
                 .cumcount() + 1
    
    # Reorder field names
    TeraNovaFieldReport_stg = TeraNovaFieldReport_stg[["ProjectID"
                                                     , "ReportScraperID" 
                                                     ,"ReportFileName"
                                                     , "ReportDateIssued"
                                                     , "ReportTimeIngested"
                                                     , "SiteTemperature"
                                                     , "SiteWeather"
                                                     , "ConcreteSupplier"
                                                     , "ConcretePlacementLocation"
                                                     , "BatchLabNumber"
                                                     , "BatchMixID"
                                                     , "BatchTimeMolded"
                                                     , "BatchSpecimenSize"
                                                     , "BatchTemperature"
                                                     , "BatchUnitWeight"
                                                     , "BatchRequiredStrength"
                                                     , "BatchAirContent"
                                                     , "BatchtSlump"
                                                     , "BatchWaterAdded"
                                                     , "Date_Molded"
                                                     , "Time_Molded"
                                                     , "BatchLabID"]]
    

    TeraNovaFieldReport_stg['LightWeight'] = 0

    
    return TeraNovaFieldReport_stg,TeraNovaTestResult_stg

#### Branson Exoport text fields

In [7]:
def extract_field_report_table(path, text, repeated,Bran_log_file,prjct_id,scrpr_id):

  
    Project_Name = Mix_id = City = Date_casted = Date_Report_Issued = Weather = Temperature_Ambient = concrete_supplier = ''
    Req_load = Location = Structural_Element = Mix_id = Building_Height = Water_Added = Admixtures_noted = specimen_size = ''
    Report_time_ingested = ''


    # Fetching the file name
    File_name = Path(path).stem

    # Data Ingestion_datetime
    Report_time_ingested = pd.to_datetime('today').strftime("%Y-%m-%d %I:%M:%S %p")


    for row in text.split('\n'):
        
        #Date & Cylinder Pick-up Date:
        if Date_casted == '':
            x_d = re.search("Date:", row)
            if x_d:
                x_pick = re.search("Pick-up Date:", row)
                if x_pick:
                    Cylinder_pick_up_date = row.split("Pick-up Date:",1)[1].strip()
                else:
                    Date_casted = row.split("Date:",1)[1].strip()
            else:
                Date_casted == ''


        #Date Report Issued & Date tested
        x_date = re.search("Date", row)
        x_D = re.search("DATE", row)
        if x_date:
            x_rep = re.search("Date Report Issued:", row)
            if x_rep:
                Report_Issued = row.split("Date Report Issued:",1)[1].strip()

                # Changing the date format:
                dt_issued = datetime.strptime(Report_Issued, '%B %d, %Y').date()
                Date_Report_Issued = dt_issued.strftime('%Y-%m-%d %H:%M:%S')
  
        elif x_D:
            Date_Tested = row.split("DATE TESTED",1)[1].strip()


        #Weather
        if Weather == '':
            x_w = re.search("Weather", row)
            if x_w:
                string_weather = row.replace(" ", "")   
                start = 'Weather:'
                end = 'Air'
                Weather = (string_weather.split(start))[1].split(end)[0].strip()
            else:
                Weather = ''

  
        #Temperature_Ambient
        if Temperature_Ambient == '':
            x = re.search("Air", row)
            if x:
                string_air = row.replace(" ", "")  

                AirTemp = re.search("AirTemperature:", string_air)
                if AirTemp:
                    Temperature_Ambient = string_air[AirTemp.end():][:-3]    
                else:
                    Temperature_Ambient = ''

        #Location of Placement
        if Location == '':
            x_loc = re.search("Locations:", row)
            if x_loc:
                Location = row.split("Locations:",1)[1].strip()                  
            else:
                Location = ''

    
        #Mix ID
        if Mix_id == '':
            x_mix = re.search("Mix", row)
            if x_mix:
                string_mix = row.replace(" ", "")
                start = 'Mix'
                end = '.'
                mixid = (string_mix.split(start))[1].split(end)[0]
                Mix_id = re.sub('\D', '', mixid)
            else:
                Mix_id = ''


        #Water Added
        if Water_Added == '':
            x_water = re.search("Water", row, re.IGNORECASE)
            if x_water:
                Water_Added = "Water found"
            else:
                Water_Added = ''

    
        #Any Admixtures noted, 
        if Admixtures_noted == '': 
            x_ad_mix = re.search("Admix", row, re.IGNORECASE)

            if x_ad_mix:
                Admixtures_noted = "Admixtures noted is found"
            else:
                Admixtures_noted = ''


        #Required Load
        if Req_load == '':
            x = re.search("CLASS", row)
            if x:
                string_req_load = row.replace(" ", "")  
                start = 'CLASS(PSI)'
                end = 'Max/Min'
                Req_load = (string_req_load.split(start))[1].split(end)[0].strip()
            else:
                Req_load = ''


        #Concrete supplier
        if concrete_supplier == '':
            x = re.search("Supplier", row)
            if x:
                string_supplier = row.replace(" ", "")  
                start = 'ConcreteSupplier:'
                end = 'CylinderPick'
                concrete_supplier = (string_supplier.split(start))[1].split(end)[0].strip()
            else:
                concrete_supplier = ''
    
 
        #Specimen size
        if specimen_size == '':
            x = re.search("Field Set", row)
  
            if x:
                string_size = row.replace(" ", "")  
                start = 'FieldSet('
                end = ')'
                size = (string_size.split(start))[1].split(end)[0].strip().replace('"', "")
                specimen_size = re.sub("[^A-Z0-9]", "", size,0,re.IGNORECASE)
            else:
                specimen_size = ''
                
        # PROJECT ID
        ProjectID = prjct_id
        
        # SCRAPER ID
        ReportScraperID = scrpr_id



  #### IMPORTING FIELDS TO A DATA FRAME ####
    Table_1_header = ["ProjectID", "ReportScraperID" ,"ReportFileName", "ReportTimeIngested", "Date_Molded", "ReportDateIssued", "SiteWeather", "SiteTemperature", "ConcreteSupplier", "BatchRequiredStrength", 
                  "ConcretePlacementLocation", "BatchMixID",  "BatchWaterAdded", "BatchAdmixturesAdded", "BatchSpecimenSize" ]

    Values = [prjct_id,scrpr_id,File_name, Report_time_ingested, Date_casted, Date_Report_Issued, Weather, Temperature_Ambient, concrete_supplier, Req_load, 
         Location, Mix_id, Water_Added, Admixtures_noted, specimen_size]

    new_row_table_1 = pd.DataFrame(Values, Table_1_header).T

  

  ##############  TABLE 2  ################


    all_tables = cam.read_pdf(path, pages = 'all', flavor = 'lattice', copy_text=['h'])

    x_table_field = []

    for i in range(all_tables.n):
        
        if 0 < i <= repeated:
            
            tab = all_tables[i].df

            #Drop unwanted columns:
            tab_new = tab.drop(labels=[1,2,3], axis=0)  

            #Converting the dataframe to string
            tab_new = tab_new.astype(str)

            # Replacing empty spaces with Null Values
            tab_new = tab_new.replace(r'^\s*$', np.nan, regex=True)

            # Dropping the columns with all Null values:
            tab_new = tab_new.dropna(axis=1, how='all')

            # Transpose
            tab_tran = tab_new.T

            # Converting first row into header:
            headers = tab_tran.iloc[0]
            tab_field = pd.DataFrame(tab_tran.values[1:], columns = headers) 

            tab_field['Field Set (4”x8”) #'] = tab_field['Field Set (4”x8”) #'].str.replace(r'[^\d.]+', '')

            # Replacing the names of first column elements to 'BatchLabNumber' and 'Density' fields:
            df_Table_2 =  tab_field
            df_Table_2 = df_Table_2.rename(columns={'Field Set (4”x8”) #': 'BatchLabNumber'})
            df_Table_2 = df_Table_2.rename(columns={'Density (lbs/cu ft)': 'Unit_Weight (lbs/cu ft)'}) 



            # Chaging the format of 'Time Discharge' and making the values NULL, for invalid time formats
            # And storing them in logfiles.
            df_Table_2['Time Discharge'] = df_Table_2['Time Discharge'].str.replace('\D', '', regex=True)     # Removind values other than digits
      
            for index in range(len(df_Table_2)):
            
                time_dis = df_Table_2.loc[index, 'Time Discharge']
        
                if len(time_dis) == 4:
                    time_conv = datetime.strptime(time_dis, '%H%M').time()
                    t = time_conv.strftime('%H:%M:%S')
                    t1 = time_conv.strftime('%I:%M %p')
                    df_Table_2.loc[index,'Time Discharge'] = t1
                    df_Table_2.loc[index, 'BatchTimeMolded'] = t

                else:
                    lab_number = df_Table_2.loc[index,'BatchLabNumber']
                    #df_Table_2.drop(df_Table_2.index[i], inplace = True)
                    df_Table_2.loc[index,'Time Discharge'] = ''                                         #replace with np.nan
                    df_Table_2.loc[index, 'BatchTimeMolded'] = ''                                     #replace with np.nan
          
                    Comment = 'Passed NULL value for Time molded for the lab no', lab_number, ' due to incorrect time format'
                    Bran_log_file = Bran_log_file.append({'File Name' : File_name, 'Comment' : Comment}, ignore_index = True)


            # Replacing the name of the columns
            df_Table_2.rename(columns={"Air Content (%)": "BatchAirContent"}, inplace=True)
            df_Table_2.rename(columns={"Slump (inches)": "BatchtSlump"}, inplace=True)
            df_Table_2.rename(columns={"Unit_Weight (lbs/cu ft)": "BatchUnitWeight"}, inplace=True)
            df_Table_2.rename(columns={"Concrete Temperature (°F)": "BatchTemperature"}, inplace=True)
            df_Table_2.rename(columns={"Time Discharge": "Time_Molded"}, inplace=True)


            if i == 1:
                x_table_field = df_Table_2
            else:
                x_table_field = pd.concat([x_table_field, df_Table_2], axis=0, ignore_index=True)

        else:
            continue


    # MERGING BOTH THE TABLES
  
    tables_merged = pd.concat([new_row_table_1, x_table_field], axis=1)

    tables_merged["ReportFileName"] = File_name 
    tables_merged["ReportTimeIngested"] = Report_time_ingested
    tables_merged["Date_Molded"] = Date_casted
    tables_merged["ReportDateIssued"] = Date_Report_Issued
    tables_merged["SiteWeather"] = Weather
    tables_merged["SiteTemperature"] = Temperature_Ambient
    tables_merged["ConcreteSupplier"] = concrete_supplier
    tables_merged["BatchRequiredStrength"] = Req_load
    tables_merged["ConcretePlacementLocation"] = Location
    tables_merged["BatchMixID"] = Mix_id
    tables_merged["BatchWaterAdded"] = Water_Added
    tables_merged["Admixtures_noted"] = Admixtures_noted
    tables_merged["BatchSpecimenSize"] = specimen_size
    tables_merged["ProjectID"] = prjct_id
    tables_merged["ReportScraperID"] = scrpr_id

   
    # Changing the date format of 'Date_Casted' field:
    dt_cas = datetime.strptime(Date_casted, '%B %d, %Y').date()
    new_date_format = dt_cas.strftime('%Y-%m-%d')

 
    # Batch_time_molded field:

    for count in range(len(tables_merged)):
        if tables_merged.loc[count, 'BatchTimeMolded'] != '':
            tables_merged.loc[count, 'BatchTimeMolded'] = new_date_format + ' ' + tables_merged.loc[count, 'BatchTimeMolded']
    
    field_report_table = tables_merged


    # Reordering the dataframe columns
    field_report_table = field_report_table[["ProjectID", "ReportScraperID" ,'ReportFileName', "ReportDateIssued", "ReportTimeIngested", "SiteTemperature", "SiteWeather", 
                      "ConcreteSupplier", "ConcretePlacementLocation", "BatchLabNumber", "BatchMixID", "BatchTimeMolded",  
                      "BatchSpecimenSize", "BatchTemperature", "BatchUnitWeight", "BatchRequiredStrength",  "BatchAirContent",
                      "BatchtSlump", "BatchWaterAdded", "BatchAdmixturesAdded", "Date_Molded", "Time_Molded"]]



    return field_report_table, Bran_log_file, Date_Report_Issued

#### Branson Extract Test Results

In [8]:
def extract_compression_table(path, text, repeated, Date_Report_Issued):
    read_all_tables = cam.read_pdf(path, pages = 'all', flavor = 'lattice', copy_text=['h'])

    x_table = []

    for i in range(read_all_tables.n):
        if i <= repeated:
            continue
        else:
            tab = read_all_tables[i].df

            #Converting the dataframe to string
            tab = tab.astype(str)

            # Replacing empty spaces with Null Values
            tab = tab.replace(r'^\s*$', np.nan, regex=True)

            # Dropping the columns with all Null values:
            tab = tab.dropna(axis=1, how='all')

            #Drop unwanted columns:
            tab_new = tab.drop(labels=[2,3,6,8,9,10,11], axis=0)
            tab_new_trans = tab_new.T

            # Converting first row into header:
            # Changing name of the first header element to 'BatchLabNumber'
            headers = tab_new_trans.iloc[0]
            tab_final = pd.DataFrame(tab_new_trans.values[1:], columns = headers)
            tab_final.columns.values[0:1] =['BatchLabNumber']

            # Changing the name of the field
            tab_final = tab_final.rename(columns={'PSI': 'Unit_Load (PSI)'})
            tab_final = tab_final.rename(columns={'Unit_Load (PSI)': 'Unit_load (lbs/sq in)'})
            tab_final['BatchLabNumber'] = tab_final['BatchLabNumber'].str.replace(r'[^\d.]+', '')

            # Dropping the rows with if any Null values:
            tab_final = tab_final.dropna()
      
            if i == repeated + 1:
                x_table = tab_final                                                  
            else:
                x_table = pd.concat([x_table, tab_final], axis=0, ignore_index=True)

  
    x_table['ReportDateIssued'] = Date_Report_Issued

    '''
      # Add a new column 'CylinderTestID':
      x_table['CylinderTestID'] = x_table.sort_values(['ReportDateIssued','TEST AT (days)'], ascending=[True,True]) \
                 .groupby(['BatchLabNumber']).cumcount() + 1


      # Replacing the name of the columns
      x_table.rename(columns={"CYLINDER #": "Cylinder_Number"}, inplace=True)
    '''

    x_table.rename(columns={"CYLINDER #": "Cylinder_Number"}, inplace=True)
    x_table.rename(columns={"DATE TESTED": "SpecimenDateTested"}, inplace=True)
    x_table.rename(columns={"TEST AT (days)": "SpecimenAgeTested"}, inplace=True)
    x_table.rename(columns={"Unit_load (lbs/sq in)": "SpecimenMeasuredStrength"}, inplace=True)
  

    compression_table = x_table

    return compression_table

#### Branson Main Function

In [9]:
def Branson(folder_dir,prjct_id,scrpr_id):\
    
    field_report_table = []
    compression_table = []
    time_format = ''
    res = ''

    # Creating a log dataframes:
    Bran_invalid_file_log = pd.DataFrame(columns=['File Name', 'Comment'])
    Bran_log_file = pd.DataFrame(columns=['File Name', 'Comment'])                       # Final log file


    for root, dirs, files in os.walk(folder_dir,topdown=True):
        for file in files:
            if file.endswith(".pdf"):
                # Skipping the files with age days other than 7 or 28 days,  and maintaining them in Log table
                res = re.findall(r'\(.*?\)', file)
                if not '7' in str(res):
                    if not '28' in str(res):
                        Bran_invalid_file_log = Bran_invalid_file_log.append({'File Name' : file, 'Comment' : 'Invalid file with Age days other than 7 or 28'}, ignore_index = True)
                        continue

                bran_path = root.replace('\\','/')+'/'+file
                pdf = pdfplumber.open(bran_path)
                page = pdf.pages[0]
                text = page.extract_text()

                # Cheking the number of times the string 'Field Set' is repeated, 
                # to count the number of 'Concrete Field Inspection and Test Results' tables:
        
                repeated = text.count("Field Set")

                # Importing data to the function
                extracted_first, Bran_log_file, Date_Report_Issued = extract_field_report_table(bran_path, text, repeated,Bran_log_file,prjct_id,scrpr_id)
                extracted_second = extract_compression_table(bran_path, text, repeated, Date_Report_Issued)

                # Appending data into tables:
                field_report_table = extracted_first.append(field_report_table, ignore_index=True)
                compression_table = extracted_second.append(compression_table, ignore_index=True)
        
                # Appendind data into log_file
                Bran_log_file = pd.concat([Bran_log_file, Bran_invalid_file_log], axis=0, ignore_index=True)

        #return name, field_report_table, compression_table, Bran_log_file
    return field_report_table, compression_table, Bran_log_file

#### Branson Data Transform

In [10]:
def Branson_data_transform(table_1, table_2, table_log, Directory):


    os.chdir(ExportDir)

    # Delteing duplicate entries from field_report_table (SpecimenFieldReport):
    Branson_FieldReportTable  = table_1[table_1.groupby('BatchLabNumber')['ReportDateIssued'].transform('max') == table_1['ReportDateIssued']].reset_index(drop=True)

    # Delteing duplicate entries from compression_table (SpecimenCompressionTestResult):
    Branson_CompressionResults  = table_2[table_2.groupby('BatchLabNumber')['ReportDateIssued'].transform('max') == table_2['ReportDateIssued']].reset_index(drop=True)


    # Add a new column CylinderTestID
    Branson_CompressionResults['CylinderTestID'] = Branson_CompressionResults.sort_values(['ReportDateIssued','SpecimenAgeTested'], ascending=[True,True]) \
    .groupby(['BatchLabNumber']).cumcount() + 1


    # generate the uuid for BatchLabID 
    
    batchlabID = {BatchLabNumber: str(uuid.uuid4()) for BatchLabNumber in Branson_FieldReportTable['BatchLabNumber'].unique()}
    Branson_FieldReportTable['BatchLabID'] = Branson_FieldReportTable['BatchLabNumber'].map(batchlabID)
    Branson_CompressionResults['BatchLabID'] = Branson_CompressionResults['BatchLabNumber'].map(batchlabID)

  
    # Changing the datatypes of all the fields from Field report table:
    Branson_FieldReportTable['ReportTimeIngested'] = Branson_FieldReportTable['ReportTimeIngested'].astype('datetime64[ns]')
    Branson_FieldReportTable['ReportDateIssued'] = pd.to_datetime(Branson_FieldReportTable["ReportDateIssued"]).dt.floor('D')
    Branson_FieldReportTable['BatchTimeMolded'] = Branson_FieldReportTable['BatchTimeMolded'].astype('datetime64[ns]')

    Branson_FieldReportTable['SiteTemperature'] = Branson_FieldReportTable['SiteTemperature'].replace(r'^\s*$', np.nan, regex=True)
    Branson_FieldReportTable['SiteTemperature'] = Branson_FieldReportTable['SiteTemperature'].astype('float')
    Branson_FieldReportTable['BatchTemperature'] = Branson_FieldReportTable['BatchTemperature'].astype('float')
    Branson_FieldReportTable['BatchUnitWeight'] = Branson_FieldReportTable['BatchUnitWeight'].astype('float')
    #Branson_FieldReportTable['BatchRequiredStrength'] = Branson_FieldReportTable['BatchRequiredStrength'].astype('float')
    Branson_FieldReportTable['BatchAirContent'] = Branson_FieldReportTable['BatchAirContent'].astype('float')
    Branson_FieldReportTable['BatchtSlump'] = Branson_FieldReportTable['BatchtSlump'].astype('float')
    #Branson_FieldReportTable['BatchWaterAdded'] = Branson_FieldReportTable['BatchWaterAdded'].astype('float')


    # Changing the datatypes of all the fields from Compression test results table:
    Branson_CompressionResults['SpecimenAgeTested'] = Branson_CompressionResults['SpecimenAgeTested'].astype('int')
    Branson_CompressionResults['ReportDateIssued'] = pd.to_datetime(Branson_CompressionResults["ReportDateIssued"]).dt.floor('D')
    Branson_CompressionResults['SpecimenMeasuredStrength'] = Branson_CompressionResults['SpecimenMeasuredStrength'].astype('int')
    Branson_CompressionResults['SpecimenDateTested'] = pd.to_datetime(Branson_CompressionResults["SpecimenDateTested"], format="%m/%d/%y").dt.floor('D')
    
    # Add another column for Light Weight
    LightWeight = []

    for idx, row in Branson_FieldReportTable.iterrows():
        x_lw = re.search('LW', row['BatchRequiredStrength'])
        
        if x_lw:
            value = row['BatchRequiredStrength'].replace('LW', '')
            Branson_FieldReportTable['BatchRequiredStrength'][idx] = value
            LightWeight.append(1)
        else:
            LightWeight.append(0)
 
    Branson_FieldReportTable['LightWeight'] = LightWeight

    # Change datatype to float
    Branson_FieldReportTable['BatchRequiredStrength'] = Branson_FieldReportTable['BatchRequiredStrength'].astype('float')


    return Branson_FieldReportTable, Branson_CompressionResults

#### Export Data Files Function

In [11]:
def ExportDataFiles(TeraNovaFieldReport_Final,TeraNovaTestResult_Final,TeraNovaFileLog,ExportDir,engineer_report,Branson_FieldReportTable,Branson_CompressionResults,final_log_file):
    
    os.chdir(ExportDir)   # Set the working directory
    
    # Exoport Updated Project Information File
    engineer_report.to_excel('ProjectInformationUpdated.xlsx', index=False)
    
    # Exporting SpecimenCompressionTestResult to excel:
    if len(Branson_CompressionResults.index) == 0 and len(TeraNovaTestResult_Final.index) == 0:
        pass
        
    elif len(Branson_CompressionResults.index) == 0 and len(TeraNovaTestResult_Final.index) != 0:
        TeraNovaTestResult_Final.to_excel('SpecimenCompressionTestResult.xlsx', index=False)
        
    elif len(Branson_CompressionResults.index) != 0 and len(TeraNovaTestResult_Final.index) == 0:
        Branson_CompressionResults.to_excel('SpecimenCompressionTestResult.xlsx', index=False)
        
    else:
        pd.concat([TeraNovaTestResult_Final
                  ,Branson_CompressionResults.drop(['Cylinder_Number'], axis=1)]
              , ignore_index=True).to_excel('SpecimenCompressionTestResult.xlsx', index=False)
    
    
     # Exporting SpecimenFieldReport data to excel:
    if len(Branson_FieldReportTable.index) == 0 and len(TeraNovaFieldReport_Final.index) == 0:
        pass
        
    elif len(Branson_FieldReportTable.index) == 0 and len(TeraNovaFieldReport_Final.index) != 0:
        TeraNovaFieldReport_Final.drop(["Date_Molded", "Time_Molded" ]
                                       , axis=1).to_excel('SpecimenFieldReport.xlsx', index=False)
        
    elif len(Branson_FieldReportTable.index) != 0 and len(TeraNovaFieldReport_Final.index) == 0:
        Branson_FieldReportTable.drop(["BatchAdmixturesAdded","Date_Molded", "Time_Molded"]
                                      , axis=1).to_excel('SpecimenFieldReport.xlsx', index=False)
    else:
        pd.concat([TeraNovaFieldReport_Final.drop(["Date_Molded", "Time_Molded" ], axis=1)
                  ,Branson_FieldReportTable.drop(["BatchAdmixturesAdded","Date_Molded", "Time_Molded"], axis=1)]
              , ignore_index=True).to_excel('SpecimenFieldReport.xlsx', index=False) 
    
    if len(final_log_file.index) == 0:
        pass
    else:
        final_log_file.to_excel(r'Branson_bad_data_log_report.xlsx', index=False)
               
#         TeraNovaFileLog.to_excel('INGESTION_LOG.xlsx', index=False)
#         final_log_file.to_excel(r'log_report.xlsx', index=False)

    return 'Excel Files Exported Successfully'



#### Export Job Log File Function

In [12]:
def JobLog(TeraNovaFileLog,TeraNovaFieldReport,TeraNovaTestResult,TeraNovaFieldReport_Final,TeraNovaTestResult_Final,jobTimeDiff,ExportDir):
    
    os.chdir(ExportDir)
    
    if len(TeraNovaFieldReport_Final.index) and len(Branson_FieldReportTable.index) != 0:
        with open('JobComplete.log', 'w') as f:
            print('========================================================================================', file=f)
            print('\t\t\t\t',date.today().strftime("%A,%d %B, %Y"), file=f)
            print('\t\t\t\t       ',datetime.now().time().strftime("%I:%M:%S %p"), file=f)
            print('========================================================================================', file=f)
            print('\t\t\t\t  TeraNova Scrapper Logs', file=f)
            print('========================================================================================', file=f)
            print('Total Files Read:', len(TeraNovaFileLog), file=f)
            print('Total Reports Read:', len(TeraNovaFieldReport), file=f)
            print('Total Test Obervations Read:', len(TeraNovaTestResult), file=f)
            print('----------------- Ingestion of Reports with Latest Reporting Date ----------------------', file=f)
            print('Total Reports Ingested:', len(TeraNovaFieldReport_Final), file=f)
            print('----------------- Ingestion of Acceptable and Latest Test Results ----------------------', file=f)
            print('Total Test Results Ingested:', len(TeraNovaTestResult_Final), file=f)
            print('========================================================================================', file=f)
            print('\t\t\t\t  Branson Scrapper Logs', file=f)
            print('========================================================================================', file=f)
            print('----------------- Ingestion of Reports with Latest Reporting Date ----------------------', file=f)
            print('Total Reports Read and Ingested:', len(Branson_FieldReportTable),file=f)
            print('----------------- Ingestion of Acceptable and Latest Test Results ----------------------', file=f)
            print('Total Test Results Read and Ingested:', len(Branson_CompressionResults) ,file=f)
            print('========================================================================================', file=f)
            print("\t\t\tTotal job took %d hours %d minutes %d seconds" % (jobTimeDiff.hours, jobTimeDiff.minutes, jobTimeDiff.seconds), file=f)
            print('========================================================================================', file=f)
        
    elif len(TeraNovaFieldReport_Final.index) == 0 and len(Branson_FieldReportTable.index) != 0:
        with open('JobComplete.log', 'w') as f:
            print('========================================================================================', file=f)
            print('\t\t\t\t',date.today().strftime("%A,%d %B, %Y"), file=f)
            print('\t\t\t\t       ',datetime.now().time().strftime("%I:%M:%S %p"), file=f)
            print('========================================================================================', file=f)
            print('========================================================================================', file=f)
            print('\t\t\t\t No TeraNova Files Read', file=f)
            print('========================================================================================', file=f)
            print('\t\t\t\t  Branson Scrapper Logs', file=f)
            print('========================================================================================', file=f)
            print('----------------- Ingestion of Reports with Latest Reporting Date ----------------------', file=f)
            print('Total Reports Read and Ingested:', len(Branson_FieldReportTable),file=f)
            print('----------------- Ingestion of Acceptable and Latest Test Results ----------------------', file=f)
            print('Total Test Results Read and Ingested:', len(Branson_CompressionResults) ,file=f)
            print('========================================================================================', file=f)
            print("\t\t\tTotal job took %d hours %d minutes %d seconds" % (jobTimeDiff.hours, jobTimeDiff.minutes, jobTimeDiff.seconds), file=f)
            print('========================================================================================', file=f)
    elif len(TeraNovaFieldReport_Final.index) != 0 and len(Branson_FieldReportTable.index) == 0:
        with open('JobComplete.log', 'w') as f:
            print('========================================================================================', file=f)
            print('\t\t\t\t',date.today().strftime("%A,%d %B, %Y"), file=f)
            print('\t\t\t\t       ',datetime.now().time().strftime("%I:%M:%S %p"), file=f)
            print('========================================================================================', file=f)
            print('\t\t\t\t  TeraNova Scrapper Logs', file=f)
            print('========================================================================================', file=f)
            print('Total Files Read:', len(TeraNovaFileLog), file=f)
            print('Total Reports Read:', len(TeraNovaFieldReport), file=f)
            print('Total Test Obervations Read:', len(TeraNovaTestResult), file=f)
            print('----------------- Ingestion of Reports with Latest Reporting Date ----------------------', file=f)
            print('Total Reports Ingested:', len(TeraNovaFieldReport_Final), file=f)
            print('----------------- Ingestion of Acceptable and Latest Test Results ----------------------', file=f)
            print('Total Test Results Ingested:', len(TeraNovaTestResult_Final), file=f)
            print('========================================================================================', file=f)
            print('\t\t\t\t No Branson Files Read', file=f)
            print('========================================================================================', file=f)
            print("\t\t\tTotal job took %d hours %d minutes %d seconds" % (jobTimeDiff.hours, jobTimeDiff.minutes, jobTimeDiff.seconds), file=f)
            print('========================================================================================', file=f)
    else:
         with open('JobComplete.log', 'w') as f:
            print('========================================================================================', file=f)
            print('\t\t\t\t',date.today().strftime("%A,%d %B, %Y"), file=f)
            print('\t\t\t\t       ',datetime.now().time().strftime("%I:%M:%S %p"), file=f)
            print('========================================================================================', file=f)
            print('\t\t\t\t No TeraNova or Branson file Read', file=f)
            print('========================================================================================', file=f)
            print("\t\t\tTotal job took %d hours %d minutes %d seconds" % (jobTimeDiff.hours, jobTimeDiff.minutes, jobTimeDiff.seconds), file=f)
            print('========================================================================================', file=f)
        
            
        
    return 'Check JobComplete.log File'

## Main Code

In [13]:
# Note Down the Job Start DateTime
jobStart = datetime.now()

# Save Export Files Directory
ExportDir = os.getcwd()

# Read the Engineer's File
engineer_report = pd.read_excel('ProjectInformation.xlsx')

# generate the uuid for ProjectID
prjctid = {DCEProjectNumber: str(uuid.uuid4()) for DCEProjectNumber in engineer_report['DCEProjectNumber'].unique()}

# map uuid
engineer_report['ProjectID'] = engineer_report['DCEProjectNumber'].map(prjctid)

# Declare TeraNovaDatframes
TeraNovaFieldReport = pd.DataFrame()
TeraNovaTestResult = pd.DataFrame()
TeraNovaFileLog = pd.DataFrame()
TeraNovaFieldReport_Final = pd.DataFrame()
TeraNovaTestResult_Final = pd.DataFrame()

# Declare Branson Dataframes
Branson_CompressionResults = pd.DataFrame()
Branson_FieldReportTable = pd.DataFrame()
final_log_file = pd.DataFrame()
final_field_report_table = pd.DataFrame()
final_compression_table = pd.DataFrame()

# Specify a varibale for max age days to consider in the data
age_day_max = 100

for index, row in engineer_report.iterrows():
    if (engineer_report['ScraperID'].isnull()[index]) & (engineer_report['Directory'].isnull()[index]):
        pass
    else:
        scrpr_id = engineer_report['ScraperID'][index]
        foldeR_dir = engineer_report['Directory'][index].replace('\\', '/')
                
        if scrpr_id == 1001:
            prjct_id = engineer_report['ProjectID'][index]
            FieldReport, TestResult, FileLog = TeraNova(foldeR_dir,prjct_id,scrpr_id)
            
            TeraNovaFieldReport = TeraNovaFieldReport.append(FieldReport).reset_index(drop=True) # Appends results to DF
            TeraNovaTestResult = TeraNovaTestResult.append(TestResult).reset_index(drop=True) # Appends results to DF
            TeraNovaFileLog = TeraNovaFileLog.append(FileLog).reset_index(drop=True) # Appends results to DF
            
        elif scrpr_id == 2001:
            scrpr_id = engineer_report['ScraperID'][index]
            foldeR_dir = engineer_report['Directory'][index].replace('\\', '/')
            prjct_id = engineer_report['ProjectID'][index]
            
            final_field_report_table, final_compression_table, final_log_file =   Branson(foldeR_dir,prjct_id,scrpr_id)
        else:
            pass

# Call TeraNova DataTransform Function
if len(TeraNovaFieldReport.index) | len(TeraNovaTestResult.index) == 0:
    pass
else:
    TeraNovaFieldReport_Final, TeraNovaTestResult_Final = TeraNovaDataTransform(TeraNovaFieldReport, TeraNovaTestResult)
   
    
# Call Branson DataTransform Function
if len(final_field_report_table.index) | len(final_compression_table.index) == 0:
    pass
else:
    Branson_FieldReportTable, Branson_CompressionResults = Branson_data_transform(final_field_report_table, final_compression_table, final_log_file, ExportDir)

# Call Export Data Files Function
ExportDataFiles(TeraNovaFieldReport_Final,TeraNovaTestResult_Final,TeraNovaFileLog,ExportDir,engineer_report,Branson_FieldReportTable,Branson_CompressionResults,final_log_file)

# Note Down the Job Finish DateTime
jobStop = datetime.now()

# Get the JobTime Difference
jobTimeDiff = relativedelta(jobStop,jobStart)

# Call the Jog Log File Function
JobLog(TeraNovaFileLog,TeraNovaFieldReport,TeraNovaTestResult,TeraNovaFieldReport_Final,TeraNovaTestResult_Final,jobTimeDiff,ExportDir)


'Check JobComplete.log File'